In [1]:
# Configurações Iniciais e Imports
import os
import sys
import logging
import time
import trimesh
import numpy as np
import matplotlib.pyplot as plt

# Adicionar o diretório raiz do projeto ao path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.core.mesh_processor import MeshProcessor
from src.utils.visualization import plot_landmarks_2d, plot_landmarks, OPEN3D_AVAILABLE
from src.utils.helpers import setup_logging

# Configurar logging
setup_logging(log_level=logging.INFO)

print(f"Open3D disponível: {OPEN3D_AVAILABLE}")
print(f"Diretório de trabalho: {os.getcwd()}")
print(f"Path do módulo: {module_path}")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Open3D disponível: True
Diretório de trabalho: d:\Users\rezen\Estudo\Unit\TCC\Projeto\landmark_detection_system\notebooks
Path do módulo: d:\Users\rezen\Estudo\Unit\TCC\Projeto\landmark_detection_system


In [2]:
# Configurar diretórios
BASE_DIR = module_path
DATA_DIR = os.path.join(BASE_DIR, 'data', 'skulls')
CACHE_DIR = os.path.join(BASE_DIR, 'data', 'cache')
RESULTS_DIR = os.path.join(BASE_DIR, 'results')

# Criar diretórios se não existirem
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

print(f"Diretório de dados: {DATA_DIR}")
print(f"Diretório de cache: {CACHE_DIR}")
print(f"Diretório de resultados: {RESULTS_DIR}")

Diretório de dados: d:\Users\rezen\Estudo\Unit\TCC\Projeto\landmark_detection_system\data\skulls
Diretório de cache: d:\Users\rezen\Estudo\Unit\TCC\Projeto\landmark_detection_system\data\cache
Diretório de resultados: d:\Users\rezen\Estudo\Unit\TCC\Projeto\landmark_detection_system\results


In [3]:
# Criar arquivo STL dummy para demonstração
dummy_stl_filename = "dummy_skull_explore.stl"
dummy_stl_path = os.path.join(DATA_DIR, dummy_stl_filename)

if not os.path.exists(dummy_stl_path):
    print(f"Criando arquivo STL dummy em {dummy_stl_path}")
    # Criar uma esfera achatada para simular um crânio
    mesh_dummy = trimesh.primitives.Sphere(radius=60, subdivisions=4)
    mesh_dummy.apply_scale([1, 1, 0.7])  # Achatar um pouco
    mesh_dummy.vertices += [0, 0, 60]  # Mover para cima
    mesh_dummy.export(dummy_stl_path)
    print(f"Arquivo STL dummy '{dummy_stl_filename}' criado.")
else:
    print(f"Usando arquivo STL existente: '{dummy_stl_filename}'")

# Arquivo alvo para análise
TARGET_STL_FILENAME = dummy_stl_filename
print(f"Arquivo alvo: {TARGET_STL_FILENAME}")

Criando arquivo STL dummy em d:\Users\rezen\Estudo\Unit\TCC\Projeto\landmark_detection_system\data\skulls\dummy_skull_explore.stl


ValueError: Couldn't produce rigid transform!

In [ ]:
# Inicializar processador
processor = MeshProcessor(data_dir=DATA_DIR, cache_dir=CACHE_DIR)

# Carregar a malha (primeira vez pode demorar mais se não houver cache)
print(f"Carregando malha: {TARGET_STL_FILENAME}")
start_time = time.time()
mesh_original = processor.load_skull(TARGET_STL_FILENAME, use_cache=True)
load_time = time.time() - start_time

if mesh_original:
    print(f"✅ Malha original carregada com sucesso em {load_time:.4f}s")
    
    # Obter estatísticas da malha
    stats = processor.get_mesh_stats(mesh_original)
    print("\n=== Estatísticas da Malha Original ===")
    print(f"Vértices: {stats['vertices']:,}")
    print(f"Faces: {stats['faces']:,}")
    print(f"Área de superfície: {stats['surface_area']:.2f} mm²")
    print(f"Centroide: [{stats['centroid'][0]:.2f}, {stats['centroid'][1]:.2f}, {stats['centroid'][2]:.2f}]")
    print(f"Extensões: [{stats['extents'][0]:.2f}, {stats['extents'][1]:.2f}, {stats['extents'][2]:.2f}]")
else:
    print(f"❌ Falha ao carregar a malha {TARGET_STL_FILENAME}")
    print("Verifique se o arquivo existe e está em formato válido.")

In [ ]:
if mesh_original:
    print("Gerando visualização da malha original...")
    
    # Salvar visualização 2D
    vis_orig_2d_path = os.path.join(RESULTS_DIR, f"{os.path.splitext(TARGET_STL_FILENAME)[0]}_original_2d.png")
    
    success = plot_landmarks(
        mesh_original, 
        landmarks_dict=None,  # Sem landmarks nesta etapa
        title=f"Malha Original: {TARGET_STL_FILENAME}", 
        use_3d=False,  # Forçar 2D para notebook
        save_path_2d=vis_orig_2d_path
    )
    
    if success and os.path.exists(vis_orig_2d_path):
        print(f"✅ Visualização 2D salva em: {vis_orig_2d_path}")
        
        # Exibir a imagem no notebook (opcional)
        from IPython.display import Image, display
        try:
            print("\n=== Visualização da Malha Original ===")
            display(Image(filename=vis_orig_2d_path))
        except:
            print("Não foi possível exibir a imagem no notebook")
    else:
        print("❌ Falha ao gerar visualização")
else:
    print("❌ Malha original não carregada, impossível visualizar.")

In [ ]:
TARGET_FACES = 2000  # Alvo de faces para simplificação

if mesh_original:
    print(f"Simplificando a malha para aproximadamente {TARGET_FACES} faces...")
    
    start_time = time.time()
    mesh_simplified = processor.simplify(
        mesh_original, 
        target_faces=TARGET_FACES, 
        use_cache=True, 
        original_filename=TARGET_STL_FILENAME
    )
    simplify_time = time.time() - start_time
    
    if mesh_simplified:
        print(f"✅ Malha simplificada com sucesso em {simplify_time:.4f}s")
        
        # Obter estatísticas da malha simplificada
        stats_simplified = processor.get_mesh_stats(mesh_simplified)
        stats_original = processor.get_mesh_stats(mesh_original)
        
        print("\n=== Comparação Original vs Simplificada ===")
        print(f"Vértices: {stats_original['vertices']:,} → {stats_simplified['vertices']:,} "
              f"({(1 - stats_simplified['vertices']/stats_original['vertices'])*100:.1f}% redução)")
        print(f"Faces: {stats_original['faces']:,} → {stats_simplified['faces']:,} "
              f"({(1 - stats_simplified['faces']/stats_original['faces'])*100:.1f}% redução)")
        
        area_preservation = (stats_simplified['surface_area'] / stats_original['surface_area']) * 100
        print(f"Preservação de área: {area_preservation:.1f}%")
        
    else:
        print("❌ Falha ao simplificar a malha. Usando malha original.")
        mesh_simplified = mesh_original
else:
    print("❌ Malha original não carregada, impossível simplificar.")
    mesh_simplified = None

In [ ]:
if mesh_simplified:
    print("Gerando visualização da malha simplificada...")
    
    vis_simpl_2d_path = os.path.join(
        RESULTS_DIR, 
        f"{os.path.splitext(TARGET_STL_FILENAME)[0]}_simplified_{TARGET_FACES}f_2d.png"
    )
    
    success = plot_landmarks(
        mesh_simplified, 
        landmarks_dict=None, 
        title=f"Malha Simplificada ({len(mesh_simplified.faces)} faces): {TARGET_STL_FILENAME}", 
        use_3d=False,
        save_path_2d=vis_simpl_2d_path
    )
    
    if success and os.path.exists(vis_simpl_2d_path):
        print(f"✅ Visualização 2D da malha simplificada salva em: {vis_simpl_2d_path}")
        
        # Exibir a imagem no notebook
        from IPython.display import Image, display
        try:
            print("\n=== Visualização da Malha Simplificada ===")
            display(Image(filename=vis_simpl_2d_path))
        except:
            print("Não foi possível exibir a imagem no notebook")
    else:
        print("❌ Falha ao gerar visualização da malha simplificada")
else:
    print("❌ Malha simplificada não disponível para visualização.")

In [ ]:
if mesh_original:
    print("\n=== Testando Desempenho do Cache ===")
    
    # Teste 1: Recarregamento da malha original
    print("Recarregando malha original (deve usar cache)...")
    start_time = time.time()
    mesh_original_cached = processor.load_skull(TARGET_STL_FILENAME, use_cache=True)
    cached_load_time = time.time() - start_time
    
    if mesh_original_cached:
        print(f"✅ Recarregamento do cache: {cached_load_time:.4f}s")
        print(f"📈 Aceleração: {load_time/cached_load_time:.1f}x mais rápido")
    else:
        print("❌ Falha ao recarregar do cache")
    
    # Teste 2: Resimplificação
    print("\nResimplificando malha (deve usar cache)...")
    start_time = time.time()
    mesh_simplified_cached = processor.simplify(
        mesh_original, 
        target_faces=TARGET_FACES, 
        use_cache=True, 
        original_filename=TARGET_STL_FILENAME
    )
    cached_simplify_time = time.time() - start_time
    
    if mesh_simplified_cached:
        print(f"✅ Resimplificação do cache: {cached_simplify_time:.4f}s")
        if 'simplify_time' in locals():
            print(f"📈 Aceleração: {simplify_time/cached_simplify_time:.1f}x mais rápido")
    else:
        print("❌ Falha ao resimplificar do cache")
    
    # Verificar consistência
    if (mesh_original_cached and mesh_simplified_cached and 
        len(mesh_original_cached.vertices) == len(mesh_original.vertices) and
        len(mesh_simplified_cached.vertices) == len(mesh_simplified.vertices)):
        print("✅ Cache está funcionando corretamente - dados consistentes")
    else:
        print("⚠️  Possível inconsistência no cache")
        
else:
    print("❌ Não é possível testar cache sem malha original")

In [ ]:
if mesh_simplified:
    print("=== Análise Geométrica Adicional ===")
    
    # Análise do comprimento das arestas
    edge_lengths = mesh_simplified.edges_unique_length
    
    print(f"\nEstatísticas das Arestas:")
    print(f"Total de arestas: {len(edge_lengths):,}")
    print(f"Comprimento médio: {np.mean(edge_lengths):.4f} mm")
    print(f"Comprimento mediano: {np.median(edge_lengths):.4f} mm")
    print(f"Desvio padrão: {np.std(edge_lengths):.4f} mm")
    print(f"Comprimento mínimo: {np.min(edge_lengths):.4f} mm")
    print(f"Comprimento máximo: {np.max(edge_lengths):.4f} mm")
    
    # Criar histograma
    plt.figure(figsize=(12, 5))
    
    # Subplot 1: Histograma das arestas
    plt.subplot(1, 2, 1)
    plt.hist(edge_lengths, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
    plt.axvline(np.mean(edge_lengths), color='red', linestyle='--', label=f'Média: {np.mean(edge_lengths):.2f}')
    plt.axvline(np.median(edge_lengths), color='green', linestyle='--', label=f'Mediana: {np.median(edge_lengths):.2f}')
    plt.title(f"Distribuição do Comprimento das Arestas\n{TARGET_STL_FILENAME}")
    plt.xlabel("Comprimento da Aresta (mm)")
    plt.ylabel("Frequência")
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Subplot 2: Distribuição dos vértices por eixo
    plt.subplot(1, 2, 2)
    vertices = mesh_simplified.vertices
    plt.scatter(vertices[:, 0], vertices[:, 1], s=1, alpha=0.6, label='Projeção XY')
    plt.xlabel("X (mm)")
    plt.ylabel("Y (mm)")
    plt.title("Distribuição dos Vértices (Vista Superior)")
    plt.axis('equal')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Salvar histograma
    hist_path = os.path.join(RESULTS_DIR, f"{os.path.splitext(TARGET_STL_FILENAME)[0]}_analysis.png")
    plt.savefig(hist_path, dpi=300, bbox_inches='tight')
    print(f"\n✅ Análise geométrica salva em: {hist_path}")
    
    # Mostrar o gráfico
    plt.show()
    
    # Análise da qualidade da malha
    print(f"\nQualidade da Malha:")
    print(f"Watertight: {mesh_simplified.is_watertight}")
    print(f"Orientação consistente: {mesh_simplified.is_winding_consistent}")
    
    # Bounding box
    bounds = mesh_simplified.bounds
    print(f"\nBounding Box:")
    print(f"X: [{bounds[0,0]:.2f}, {bounds[1,0]:.2f}] mm (largura: {bounds[1,0]-bounds[0,0]:.2f} mm)")
    print(f"Y: [{bounds[0,1]:.2f}, {bounds[1,1]:.2f}] mm (profundidade: {bounds[1,1]-bounds[0,1]:.2f} mm)")
    print(f"Z: [{bounds[0,2]:.2f}, {bounds[1,2]:.2f}] mm (altura: {bounds[1,2]-bounds[0,2]:.2f} mm)")
    
else:
    print("❌ Malha simplificada não disponível para análise adicional")

In [ ]:
print("=== CONCLUSÕES DA EXPLORAÇÃO ===")
print()
print("Este notebook demonstrou o carregamento, visualização e simplificação")
print("de um modelo de crânio STL usando as ferramentas desenvolvidas no projeto.")
print()
print("### Principais Resultados:")
print()
print("1. **Carregamento eficiente**: O sistema de cache acelera significativamente operações repetidas")
print("2. **Simplificação controlada**: A decimação quadrática preserva as características importantes da malha")
print("3. **Visualização clara**: As projeções 2D permitem análise visual em qualquer ambiente")
print("4. **Análise quantitativa**: Estatísticas detalhadas sobre a geometria da malha")
print()
print("### Próximos Passos:")
print()
print("- A malha simplificada está pronta para detecção de landmarks")
print("- O sistema de cache otimizará processamentos futuros")
print("- As visualizações ajudam a validar a qualidade do pré-processamento")
print()
print("**✅ A etapa de pré-processamento foi validada como funcional e eficiente.**")